In [1]:

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam



In [4]:
# Step 1: Data Preparation
train_dir = "D:/360DigitMG/Ptoject-2/data/drowsy.v1i.folder/train"
validation_dir = "D:/360DigitMG/Ptoject-2/data/drowsy.v1i.folder/valid"

In [5]:
# Step 2: Data Preprocessing
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)


validation_datagen = ImageDataGenerator(rescale=1.0/255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)


Found 7006 images belonging to 2 classes.
Found 1999 images belonging to 2 classes.


In [6]:
# Step 3: Build the Model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

In [7]:

# Step 4: Compile the Model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [8]:



# Step 5: Train the Model
history = model.fit(
    train_generator,
    steps_per_epoch=100,  # Adjust based on your dataset size
    epochs=10,
    validation_data=validation_generator,
    validation_steps=50   # Adjust based on your dataset size
)



Epoch 1/10
100/100 [==============================] - 189s 2s/step - loss: 0.5700 - accuracy: 0.6988 - val_loss: 0.3016 - val_accuracy: 0.8906
Epoch 2/10
100/100 [==============================] - 155s 2s/step - loss: 0.2726 - accuracy: 0.8850 - val_loss: 0.2767 - val_accuracy: 0.9025
Epoch 3/10
100/100 [==============================] - 140s 1s/step - loss: 0.1797 - accuracy: 0.9322 - val_loss: 0.1259 - val_accuracy: 0.9544
Epoch 4/10
100/100 [==============================] - 133s 1s/step - loss: 0.1284 - accuracy: 0.9544 - val_loss: 0.0852 - val_accuracy: 0.9744
Epoch 5/10
100/100 [==============================] - 135s 1s/step - loss: 0.0604 - accuracy: 0.9822 - val_loss: 0.0176 - val_accuracy: 0.9937
Epoch 6/10
100/100 [==============================] - 129s 1s/step - loss: 0.0574 - accuracy: 0.9803 - val_loss: 0.0260 - val_accuracy: 0.9906
Epoch 7/10
100/100 [==============================] - 128s 1s/step - loss: 0.0496 - accuracy: 0.9850 - val_loss: 0.0524 - val_accuracy: 0.9837

In [9]:
# Step 6: Evaluate the Model
loss, accuracy = model.evaluate(validation_generator)
print(f'Validation accuracy: {accuracy*100:.2f}%')

# Optionally, you can save the model
model.save('drowsiness_detection_model.h5')


63/63 [==============================] - 15s 228ms/step - loss: 0.0104 - accuracy: 0.9955
Validation accuracy: 99.55%


C:\Users\MK PERUMALLA\anaconda3\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [10]:
from tensorflow.keras.models import load_model

# Load the model
loaded_model = load_model('drowsiness_detection_model.h5')

# Verify that the model has been loaded correctly
loss, accuracy = loaded_model.evaluate(validation_generator)
print(f'Validation accuracy: {accuracy*100:.2f}%')


63/63 [==============================] - 15s 226ms/step - loss: 0.0104 - accuracy: 0.9955
Validation accuracy: 99.55%


In [ ]:
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Load the model
model = load_model('drowsiness_detection_model.h5')

In [20]:


def preprocess_image(image_path):
    # Load the image
    img = load_img(image_path, target_size=(150, 150))  # Resize to match the input size expected by the model
    # Convert the image to array
    img_array = img_to_array(img)
    # Expand dimensions to match the input shape (1, 150, 150, 3)
    img_array = np.expand_dims(img_array, axis=0)
    # Rescale the image (same as during training)
    img_array =  img_array / 255.0
    return img_array

def predict_image(image_path):
    # Preprocess the image
    img_array = preprocess_image(image_path)
    # Make prediction
    prediction = model.predict(img_array)
    # The output is a probability, convert it to class label
    if prediction[0] > 0.5:
        return 'non-drowsy'
    else:
        return 'drowsy'

# Example usage
image_path = "D:\MY PICS\WhatsApp Image 2023-12-20 at 22.22.46_172bc2cb.jpg"
result = predict_image(image_path)
print(f'The predicted class for the image is: {result}')


1/1 [==============================] - 0s 312ms/step
The predicted class for the image is: drowsy
